# Modeling - ethereum - v5 - Comparação

# Setup

## Library import
We import all the required Python libraries

In [14]:
import os
import time

# Data manipulation
from feature_engine.encoding import RareLabelEncoder, CountFrequencyEncoder
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np

# Visualizations
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
plotly.offline.init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from lightgbm import (
    LGBMClassifier, plot_importance, create_tree_digraph, plot_tree
)
import missingno as msno
# from pycaret.classification import ClassificationExperiment
from sklearn.experimental import enable_halving_search_cv
from sklearn.feature_selection import RFECV
from sklearn.model_selection import (
    train_test_split, RandomizedSearchCV, GridSearchCV, HalvingGridSearchCV, cross_validate, KFold
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_auc_score, roc_curve,
    accuracy_score, precision_score, recall_score, f1_score
)
from sklearn import set_config

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

# Options for pandas
set_config(transform_output = "pandas")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.5f}'.format)
# pd.options.display.float_format = '{:.5f}'.format
# pd.options.display.max_rows = 120

sns.set_context(context='paper', font_scale=2, rc=None)
sns.set_style("ticks")
sns.set_palette(sns.color_palette())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Local library import
We import all the required local libraries libraries

In [15]:
os.chdir('../')
from src.utils.data_describe import breve_descricao, serie_nulos, cardinalidade, check_for_equal_columns
os.chdir('./notebooks/')

def round_4(x):
    return x.round(4)

def save_dict_to_txt(content: dict, file_path: str, mode: str = 'w', encoding: str = 'utf-8') -> None:
    try:
        with open(file_path, mode=mode, encoding=encoding) as f:
            for key, value in content.items():
                f.write(f"{key}: {value}\n")
        print(f"Dictionary successfully saved to {file_path}")
    except Exception as e:
        print(f"Error saving file: {str(e)}")

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [16]:
RAW_FOLDER = '../data/raw/'
INTERIM_FOLDER = '../data/interim/'
PROCESSED_FOLDER = '../data/processed/'
REPORTS_FOLDER = '../reports/'
RANDOM_STATE = 42

GENERATE_REPORTS = False

dct_active_models = {
    'lightgbm': True,
    'random_forest': False,
    'decision_tree': False,
    'knn': False,
    'catboost': False,
    'xgboost': False,
}

train_size = 0.15
scoring_metric = ['roc_auc', 'f1', 'recall']
refit_metric = 'roc_auc'


# palette = sns.color_palette("Spectral", as_cmap=True)
palette = sns.color_palette("husl", 10)
palette

[(0.9677975592919913, 0.44127456009157356, 0.5358103155058701),
 (0.8616090647292522, 0.536495730113334, 0.19548899031476086),
 (0.6804189127793346, 0.6151497514677574, 0.19405452111445337),
 (0.46810256823426105, 0.6699492535792404, 0.1928958739904499),
 (0.20125317221201128, 0.6907920815379026, 0.47966761189275336),
 (0.21044753832183283, 0.6773105080456748, 0.6433941168468681),
 (0.2197995660828324, 0.6625157876850336, 0.7732093159317209),
 (0.433280341176423, 0.6065273407962815, 0.9585467098271748),
 (0.8004936186423958, 0.47703363533737203, 0.9579547196007522),
 (0.9622723935096688, 0.3976451968965351, 0.8008274363432775)]

# Experience 01 - Only numerical attributes

## Data import

In [17]:
df_train_exp_01 = pd.read_parquet(INTERIM_FOLDER + 'artigo_df_train_exp_01.pqt')
df_test_exp_01 = pd.read_parquet(INTERIM_FOLDER + 'artigo_df_test_exp_01.pqt')

display(df_train_exp_01.head(3))

,Address,FLAG,Avg_min_between_sent_tnx,Avg_min_between_received_tnx,Time_Diff_between_first_and_last_(Mins),Sent_tnx,Received_Tnx,Number_of_Created_Contracts,Unique_Received_From_Addresses,Unique_Sent_To_Addresses,min_value_received,max_value_received,avg_val_received,min_val_sent,max_val_sent,avg_val_sent,min_value_sent_to_contract,max_val_sent_to_contract,avg_value_sent_to_contract,total_transactions_(including_tnx_to_create_contract,total_Ether_sent,total_ether_received,total_ether_sent_contracts,total_ether_balance,Total_ERC20_tnxs,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_total_Ether_sent_contract,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_sent_addr.1,ERC20_uniq_rec_contract_addr,ERC20_min_val_rec,ERC20_max_val_rec,ERC20_avg_val_rec,ERC20_min_val_sent,ERC20_max_val_sent,ERC20_avg_val_sent,ERC20_uniq_sent_token_name,ERC20_uniq_rec_token_name
1,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07000,2958.44000,1218216.73000,94,8,0,5,14,0.00000,2.61327,0.38569,0.00000,1.80000,0.03284,0.00000,0.00000,0.00000,102,3.08730,3.08548,0.00000,-0.00182,8.00000,403.42831,2.26081,0.00000,1.00000,5.00000,0.00000,7.00000,0.00000,365.00000,57.63262,2.26081,2.26081,2.26081,1.00000,7.00000
2,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54000,2434.02000,516729.30000,2,10,0,10,2,0.11312,1.16545,0.35891,0.05000,3.53862,1.79431,0.00000,0.00000,0.00000,12,3.58862,3.58906,0.00000,0.00044,8.00000,521.51207,0.00000,0.00000,0.00000,7.00000,0.00000,8.00000,0.00000,442.81984,65.18901,0.00000,0.00000,0.00000,0.00000,8.00000
4,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61000,10707.77000,382472.42000,4598,20,1,7,19,0.00000,12.80241,2.67110,0.00000,9.00000,0.02269,0.00000,0.00000,0.00000,4619,104.31888,53.42190,0.00000,-50.89699,42.00000,162829.66090,123539.93290,0.00000,4.00000,23.00000,0.00000,27.00000,0.00000,90000.00000,4934.23215,0.00000,45000.00000,13726.65922,6.00000,27.00000


# Categorical features encoded by frequency

## Data import

In [18]:
df_raw = pd.read_parquet(INTERIM_FOLDER + 'transaction_dataset.pqt')

df_train_exp_02 = pd.read_parquet(INTERIM_FOLDER + 'artigo_df_cleaned_train_exp_02_transformed.pqt')
df_test_exp_02 = pd.read_parquet(INTERIM_FOLDER + 'artigo_df_cleaned_test_exp_02_transformed.pqt')

X_train_exp_02 = df_train_exp_02.drop(columns=['FLAG'])
y_train_exp_02 = df_train_exp_02[['FLAG']]

X_test_exp_02 = df_test_exp_02.drop(columns=['FLAG'])
y_test_exp_02 = df_test_exp_02[['FLAG']]

print(f"""
X_train_exp_02: {X_train_exp_02.shape}
y_train_exp_02: {y_train_exp_02.shape}

X_test_exp_02: {X_test_exp_02.shape}
y_test_exp_02: {y_test_exp_02.shape}
""")

display(X_train_exp_02.head(3))


X_train_exp_02: (8343, 40)
y_train_exp_02: (8343, 1)

X_test_exp_02: (1473, 40)
y_test_exp_02: (1473, 1)



,Avg_min_between_sent_tnx,Avg_min_between_received_tnx,Time_Diff_between_first_and_last_(Mins),Sent_tnx,Received_Tnx,Number_of_Created_Contracts,Unique_Received_From_Addresses,Unique_Sent_To_Addresses,min_value_received,max_value_received,avg_val_received,min_val_sent,max_val_sent,avg_val_sent,min_value_sent_to_contract,max_val_sent_to_contract,avg_value_sent_to_contract,total_transactions_(including_tnx_to_create_contract,total_Ether_sent,total_ether_received,total_ether_sent_contracts,total_ether_balance,Total_ERC20_tnxs,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_total_Ether_sent_contract,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_sent_addr.1,ERC20_uniq_rec_contract_addr,ERC20_min_val_rec,ERC20_max_val_rec,ERC20_avg_val_rec,ERC20_min_val_sent,ERC20_max_val_sent,ERC20_avg_val_sent,ERC20_uniq_sent_token_name,ERC20_uniq_rec_token_name,ERC20_most_sent_token_type,ERC20_most_rec_token_type
2859,163.07000,0.17000,326.47000,2,2,0,2,2,35.49665,65.50334,50.50000,3.00000,97.99902,50.49951,0.00000,0.00000,0.00000,4,100.99902,101.00000,0.00000,0.00098,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.84238,0.53506
2149,0.00000,4.64000,2726.18000,1,2,0,1,1,0.01500,6.90985,3.46242,6.92436,6.92436,6.92436,0.00000,0.00000,0.00000,3,6.92436,6.92485,0.00000,0.00049,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.84238,0.53506
2979,1050.50000,887.86000,1393223.92000,689,754,0,405,448,0.00000,81.82880,1.66417,0.00000,59.98000,1.52056,0.00000,0.00000,0.00000,1443,1047.66474,1254.78214,0.00000,207.11740,31.00000,2547558.02500,387.73596,0.00000,1.00000,19.00000,0.00000,23.00000,0.00000,2537935.00000,87846.82844,0.70000,387.03596,193.86798,1.00000,23.00000,0.15762,0.46494


In [19]:
df_raw = pd.read_parquet(INTERIM_FOLDER + 'transaction_dataset.pqt')

df_train_exp_02 = pd.read_parquet(INTERIM_FOLDER + 'artigo_df_cleaned_train_exp_02_transformed.pqt')
df_test_exp_02 = pd.read_parquet(INTERIM_FOLDER + 'artigo_df_cleaned_test_exp_02_transformed.pqt')

X_train_exp_02 = df_train_exp_02.drop(columns=['FLAG'])
y_train_exp_02 = df_train_exp_02[['FLAG']]

X_test_exp_02 = df_test_exp_02.drop(columns=['FLAG'])
y_test_exp_02 = df_test_exp_02[['FLAG']]

print(f"""
X_train_exp_02: {X_train_exp_02.shape}
y_train_exp_02: {y_train_exp_02.shape}

X_test_exp_02: {X_test_exp_02.shape}
y_test_exp_02: {y_test_exp_02.shape}
""")

display(X_train_exp_02.head(3))


X_train_exp_02: (8343, 40)
y_train_exp_02: (8343, 1)

X_test_exp_02: (1473, 40)
y_test_exp_02: (1473, 1)



,Avg_min_between_sent_tnx,Avg_min_between_received_tnx,Time_Diff_between_first_and_last_(Mins),Sent_tnx,Received_Tnx,Number_of_Created_Contracts,Unique_Received_From_Addresses,Unique_Sent_To_Addresses,min_value_received,max_value_received,avg_val_received,min_val_sent,max_val_sent,avg_val_sent,min_value_sent_to_contract,max_val_sent_to_contract,avg_value_sent_to_contract,total_transactions_(including_tnx_to_create_contract,total_Ether_sent,total_ether_received,total_ether_sent_contracts,total_ether_balance,Total_ERC20_tnxs,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_total_Ether_sent_contract,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_sent_addr.1,ERC20_uniq_rec_contract_addr,ERC20_min_val_rec,ERC20_max_val_rec,ERC20_avg_val_rec,ERC20_min_val_sent,ERC20_max_val_sent,ERC20_avg_val_sent,ERC20_uniq_sent_token_name,ERC20_uniq_rec_token_name,ERC20_most_sent_token_type,ERC20_most_rec_token_type
2859,163.07000,0.17000,326.47000,2,2,0,2,2,35.49665,65.50334,50.50000,3.00000,97.99902,50.49951,0.00000,0.00000,0.00000,4,100.99902,101.00000,0.00000,0.00098,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.84238,0.53506
2149,0.00000,4.64000,2726.18000,1,2,0,1,1,0.01500,6.90985,3.46242,6.92436,6.92436,6.92436,0.00000,0.00000,0.00000,3,6.92436,6.92485,0.00000,0.00049,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.84238,0.53506
2979,1050.50000,887.86000,1393223.92000,689,754,0,405,448,0.00000,81.82880,1.66417,0.00000,59.98000,1.52056,0.00000,0.00000,0.00000,1443,1047.66474,1254.78214,0.00000,207.11740,31.00000,2547558.02500,387.73596,0.00000,1.00000,19.00000,0.00000,23.00000,0.00000,2537935.00000,87846.82844,0.70000,387.03596,193.86798,1.00000,23.00000,0.15762,0.46494


## Configurando modelos e dicionários de modelos

In [21]:
max_depth = np.arange(3, 36, 5).tolist(),
num_leaves = np.arange(2**np.min(max_depth), 70, 5).tolist()

lightgbm = LGBMClassifier(boosting_type='gbdt', random_state=RANDOM_STATE, n_jobs=-1, objective='binary', importance_type='gain', verbosity=-1)
catboost = CatBoostClassifier( random_state=RANDOM_STATE, verbose=0, early_stopping_rounds=20)
xgboost = XGBClassifier(
    random_state=RANDOM_STATE, objective='binary:logistic', eval_metric='logloss', early_stopping_rounds=10, n_jobs=-1, verbose_eval=None
)
random_forest = RandomForestClassifier(
    max_features='sqrt', bootstrap=True, n_jobs=-1, random_state=RANDOM_STATE, verbose=0
)
decision_tree = DecisionTreeClassifier(random_state=RANDOM_STATE)
knn = KNeighborsClassifier(n_jobs=-1)

dct_models = {
    'lightgbm': {
        'model': lightgbm,
        'is_active': dct_active_models['lightgbm'],
        'dct_params': dict(
            max_depth = [3, 8, 13, 18, 23, 28, 33],
            num_leaves = [8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68],
            learning_rate = [0.0001, 0.001, 0.01, 0.1],
            n_estimators = np.arange(50, 600, 100).tolist(),
        )
    },
    'random_forest': {
        'model': random_forest,
        'is_active': dct_active_models['random_forest'],
        'dct_params': dict(
            max_depth = [3, 8, 13, 18, 23, 28, 33],
            max_leaf_nodes = [8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68],
            n_estimators = np.arange(50, 600, 100).tolist(),
        )
    },
    'decision_tree': {
        'model': decision_tree,
        'is_active': dct_active_models['decision_tree'],
        'dct_params': dict(
            max_depth = [3, 8, 13, 18, 23, 28, 33],
            max_leaf_nodes = [8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68]
        )
    },
    'knn': {
        'model': knn,
        'is_active': dct_active_models['knn'],
        'dct_params': dict(
            n_neighbors = [3, 8, 13, 18, 23, 28, 33]
        )
    },
    'catboost': {
        'model': catboost,
        'is_active': dct_active_models['catboost'],
        'dct_params': dict(
            depth = [4, 6, 8, 10],
            learning_rate = [0.001, 0.01, 0.1],
            iterations = [100, 200, 300],
            l2_leaf_reg = [1, 3, 5],
            border_count = [32, 64, 128],
            random_strength = [0.1, 1, 10]
        )
    },
    'xgboost': {
        'model': xgboost,
        'is_active': dct_active_models['xgboost'],
        'dct_params': dict(
            max_depth = np.arange(3, 16, 4).tolist(),
            learning_rate = [0.0001, 0.001, 0.01, 0.1],
            n_estimators = np.arange(50, 600, 100).tolist(),
            reg_lambda = [0.1, 1, 3, 5, 10]
        )
    },
}

for key, item in dct_models.items():
    combinations = 1
    for param, values in item['dct_params'].items():
        combinations *= len(values)
    dct_models[key].update({'combinations': combinations})
    print(f"""{key}:
    dct_params: {item['dct_params']},
    combinations: {combinations}
    \n""")
    print(30*'-')

if GENERATE_REPORTS:
    save_dict_to_txt(content=dct_models, file_path=REPORTS_FOLDER+f'random_search-params_combinations-{scoring_metric}.txt')
else:
    print(f'GENERATE_REPORTS: {GENERATE_REPORTS}')


lightgbm:
    dct_params: {'max_depth': [3, 8, 13, 18, 23, 28, 33], 'num_leaves': [8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68], 'learning_rate': [0.0001, 0.001, 0.01, 0.1], 'n_estimators': [50, 150, 250, 350, 450, 550]},
    combinations: 2184
    

------------------------------
random_forest:
    dct_params: {'max_depth': [3, 8, 13, 18, 23, 28, 33], 'max_leaf_nodes': [8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68], 'n_estimators': [50, 150, 250, 350, 450, 550]},
    combinations: 546
    

------------------------------
decision_tree:
    dct_params: {'max_depth': [3, 8, 13, 18, 23, 28, 33], 'max_leaf_nodes': [8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68]},
    combinations: 91
    

------------------------------
knn:
    dct_params: {'n_neighbors': [3, 8, 13, 18, 23, 28, 33]},
    combinations: 7
    

------------------------------
catboost:
    dct_params: {'depth': [4, 6, 8, 10], 'learning_rate': [0.001, 0.01, 0.1], 'iterations': [100, 200, 300], 'l2_leaf_reg': [

## Random search

In [22]:
dct_hyperparams = {}

for model in dct_models.keys():
    if dct_models[model]['is_active']:
        try:
            print(f'Iniciar {model}.')
            start_time = time.time()
            dct_params = dct_models[model]['dct_params']
            classifier = dct_models[model]['model']

            rscv = RandomizedSearchCV(
                classifier, dct_params, error_score='raise', random_state=RANDOM_STATE, n_iter=100,
                return_train_score=True, scoring=scoring_metric, cv=3, verbose=False, refit=refit_metric
            )

            if model == 'xgboost':
                search = rscv.fit(X_train_exp_02, y_train_exp_02['FLAG'].values, eval_set=[(X_test_exp_02, y_test_exp_02)])
            else:
                search = rscv.fit(X_train_exp_02, y_train_exp_02['FLAG'])

            dct_hyperparams[model] = {
                'best_rscv': search.best_params_,
                'results': rscv.cv_results_
                }
            end_time = time.time()
            execution_time = end_time - start_time
            print(f'O modelo {model} passou pela busca aleatória em {int(execution_time//60)} minutos e {execution_time%60:.1f} segundos.\n')
        except:
            print(f'Erro ao executar o modelo {model}.\n')

    else:
        print(f'O modelo {model} está desativado.\n')


Iniciar lightgbm.
O modelo lightgbm passou pela busca aleatória em 2 minutos e 23.6 segundos.

O modelo random_forest está desativado.

O modelo decision_tree está desativado.

O modelo knn está desativado.

O modelo catboost está desativado.

O modelo xgboost está desativado.



In [23]:
df_search_results = pd.DataFrame()

for model in dct_hyperparams.keys():
    print(model)
    df_random_search_results = pd.DataFrame.from_dict(dct_hyperparams[model]['results'], orient='columns')

    df_random_search_results.insert(0, 'model', model)

    df_search_results = pd.concat([df_search_results, df_random_search_results], axis=0, ignore_index=True)

df_search_results.sort_values(by=['model', 'rank_test_roc_auc'], inplace=True, ascending=True)
display(df_search_results.loc[df_search_results['rank_test_roc_auc'] <= 3])

if GENERATE_REPORTS:
    df_search_results.to_excel(REPORTS_FOLDER + f'artigo_random_search-{scoring_metric}.xlsx')
else:
    print(f'GENERATE_REPORTS: {GENERATE_REPORTS}')

lightgbm


,model,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_num_leaves,param_n_estimators,param_max_depth,param_learning_rate,params,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_train_roc_auc,split1_train_roc_auc,split2_train_roc_auc,mean_train_roc_auc,std_train_roc_auc,split0_test_f1,split1_test_f1,split2_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_train_f1,split1_train_f1,split2_train_f1,mean_train_f1,std_train_f1,split0_test_recall,split1_test_recall,split2_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_train_recall,split1_train_recall,split2_train_recall,mean_train_recall,std_train_recall
33,lightgbm,0.24853,0.08223,0.02371,0.00100,18,250,23,0.10000,"{'num_leaves': 18, 'n_estimators': 250, 'max_depth': 23, 'learning_rate': 0.1}",0.99872,0.99912,0.99814,0.99866,0.00040,1,1.00000,1.00000,1.00000,1.00000,0.00000,0.97204,0.97792,0.96895,0.97297,0.00372,4,1.00000,1.00000,1.00000,1.00000,0.00000,0.95786,0.96921,0.96110,0.96272,0.00477,6,1.00000,1.00000,1.00000,1.00000,0.00000
92,lightgbm,0.37427,0.09806,0.03254,0.00361,23,250,18,0.10000,"{'num_leaves': 23, 'n_estimators': 250, 'max_depth': 18, 'learning_rate': 0.1}",0.99859,0.99897,0.99783,0.99846,0.00047,2,1.00000,1.00000,1.00000,1.00000,0.00000,0.97288,0.97959,0.96648,0.97298,0.00536,2,1.00000,1.00000,1.00000,1.00000,0.00000,0.95948,0.97245,0.95786,0.96326,0.00653,4,1.00000,1.00000,1.00000,1.00000,0.00000
24,lightgbm,0.20870,0.02626,0.02001,0.00116,43,150,8,0.10000,"{'num_leaves': 43, 'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.1}",0.99857,0.99876,0.99794,0.99843,0.00035,3,1.00000,1.00000,1.00000,1.00000,0.00000,0.96880,0.97465,0.96321,0.96889,0.00467,20,1.00000,1.00000,1.00000,1.00000,0.00000,0.95624,0.96596,0.95462,0.95894,0.00501,17,1.00000,1.00000,1.00000,1.00000,0.00000


GENERATE_REPORTS: False


## Grid search

In [24]:
for key, item in dct_hyperparams.items():
    print(f"{key}: {item['best_rscv']}")
    print()

lightgbm: {'num_leaves': 18, 'n_estimators': 250, 'max_depth': 23, 'learning_rate': 0.1}



In [25]:
dct_best_hyperparams = {
    'lightgbm': {
        'model': lightgbm,
        'is_active': dct_active_models['lightgbm'],
        'dct_params': dict(
            max_depth = np.arange(dct_hyperparams['lightgbm']['best_rscv']['max_depth'] - 2, dct_hyperparams['lightgbm']['best_rscv']['max_depth'] + 2, 1).tolist(),
            num_leaves = np.arange(dct_hyperparams['lightgbm']['best_rscv']['num_leaves'] - 2, dct_hyperparams['lightgbm']['best_rscv']['num_leaves'] + 2, 1).tolist(),
            learning_rate = np.arange(dct_hyperparams['lightgbm']['best_rscv']['learning_rate'] - 0.01, dct_hyperparams['lightgbm']['best_rscv']['learning_rate'] + 0.01, 0.005).tolist(),
            n_estimators = np.arange(np.max([dct_hyperparams['lightgbm']['best_rscv']['n_estimators'] - 50, 0]), dct_hyperparams['lightgbm']['best_rscv']['n_estimators'] + 100, 50).tolist(),
        )
    },
    'random_forest': {
        'model': random_forest,
        'is_active': dct_active_models['random_forest'],
        'dct_params': dict(
            max_depth = np.arange(dct_hyperparams['random_forest']['best_rscv']['max_depth'] - 2, dct_hyperparams['random_forest']['best_rscv']['max_depth'] + 2, 1).tolist(),
            max_leaf_nodes = np.arange(dct_hyperparams['random_forest']['best_rscv']['max_leaf_nodes'] - 2, dct_hyperparams['random_forest']['best_rscv']['max_leaf_nodes'] + 2, 1).tolist(),
            n_estimators = np.arange(np.max([dct_hyperparams['random_forest']['best_rscv']['n_estimators'] - 50, 0]), dct_hyperparams['random_forest']['best_rscv']['n_estimators'] + 100, 50).tolist(),
        )
    },
    'decision_tree': {
        'model': decision_tree,
        'is_active': dct_active_models['decision_tree'],
        'dct_params': dict(
            max_depth = np.arange(dct_hyperparams['decision_tree']['best_rscv']['max_depth'] - 2, dct_hyperparams['decision_tree']['best_rscv']['max_depth'] + 2, 1).tolist(),
            max_leaf_nodes = np.arange(dct_hyperparams['decision_tree']['best_rscv']['max_leaf_nodes'] - 2, dct_hyperparams['decision_tree']['best_rscv']['max_leaf_nodes'] + 2, 1).tolist(),
        )
    },
    'knn': {
        'model': knn,
        'is_active': dct_active_models['knn'],
        'dct_params': dict(
            n_neighbors = np.arange(dct_hyperparams['knn']['best_rscv']['n_neighbors'] - 2, dct_hyperparams['knn']['best_rscv']['n_neighbors'] + 2, 1).tolist()
        )
    },
    'catboost': {
        'model': catboost,
        'is_active': dct_active_models['catboost'],
        'dct_params': dict(
            depth = np.arange(dct_hyperparams['catboost']['best_rscv']['depth']-2, dct_hyperparams['catboost']['best_rscv']['depth']+2, 1).tolist(),
            learning_rate = np.arange(dct_hyperparams['catboost']['best_rscv']['learning_rate'] - 0.01, dct_hyperparams['catboost']['best_rscv']['learning_rate'] + 0.01, 0.005).tolist(),
            iterations = np.arange(np.max([dct_hyperparams['catboost']['best_rscv']['iterations'] - 50, 0]), dct_hyperparams['catboost']['best_rscv']['iterations'] + 100, 50).tolist(),
            l2_leaf_reg = np.arange(dct_hyperparams['catboost']['best_rscv']['l2_leaf_reg'] - 1, dct_hyperparams['catboost']['best_rscv']['l2_leaf_reg'] + 1, 0.5).tolist(),
            # border_count = np.arange(dct_hyperparams['catboost']['best_rscv']['border_count'] - 1, dct_hyperparams['catboost']['best_rscv']['border_count'] + 1, 1).tolist(),
            # random_strength = np.arange(dct_hyperparams['catboost']['best_rscv']['random_strength'] - 0.1, dct_hyperparams['catboost']['best_rscv']['random_strength'] + 0.1, 0.05).tolist(),
        )
    },
    'xgboost': {
        'model': xgboost,
        'is_active': dct_active_models['xgboost'],
        'dct_params': dict(
            max_depth = np.arange(dct_hyperparams['xgboost']['best_rscv']['max_depth'] - 2, dct_hyperparams['xgboost']['best_rscv']['max_depth'] + 2, 1).tolist(),
            learning_rate = np.arange(dct_hyperparams['xgboost']['best_rscv']['learning_rate'] - 0.01, dct_hyperparams['xgboost']['best_rscv']['learning_rate'] + 0.01, 0.005).tolist(),
            n_estimators = np.arange(np.max([dct_hyperparams['xgboost']['best_rscv']['n_estimators'] - 50, 0]), dct_hyperparams['xgboost']['best_rscv']['n_estimators'] + 100, 50).tolist(),
            reg_lambda = np.arange(dct_hyperparams['xgboost']['best_rscv']['reg_lambda'] - 1, dct_hyperparams['xgboost']['best_rscv']['reg_lambda'] + 1, 0.5).tolist(),
        )
    },
}

for key, item in dct_best_hyperparams.items():
    if dct_best_hyperparams[model]['is_active']:
        combinations = 1
        for param, values in item['dct_params'].items():
            combinations *= len(values)
        dct_best_hyperparams[key].update({'combinations': combinations})
        print(f"""{key}:

        dct_params: {item['dct_params']}

        combinations: {combinations}
            """)
    else:
        print(f"{key} is not active.\n")
    print(30*'-')

if GENERATE_REPORTS:
    save_dict_to_txt(content=dct_best_hyperparams, file_path=REPORTS_FOLDER+f'grid_search-params_combinations-{scoring_metric}.txt')
else:
    print(f'GENERATE_REPORTS: {GENERATE_REPORTS}')

KeyError: 'random_forest'

In [ ]:
dct_final_hyperparams = {}

for model in dct_best_hyperparams.keys():
    if dct_best_hyperparams[model]['is_active']:
        print(f'Iniciar {model}.')
        start_time = time.time()
        dct_params = dct_best_hyperparams[model]['dct_params']
        classifier = dct_best_hyperparams[model]['model']

        gscv = GridSearchCV(
            classifier, dct_params, error_score='raise', n_jobs=-1,
            return_train_score=True, scoring=scoring_metric, cv=3, verbose=False, refit=refit_metric
        )

        if model == 'xgboost':
            search = gscv.fit(X_train_exp_02, y_train_exp_02['FLAG'].values, eval_set=[(X_test_exp_02, y_test_exp_02)])
        else:
            search = gscv.fit(X_train_exp_02, y_train_exp_02['FLAG'])

        dct_final_hyperparams[model] = {
            'best_gscv': search.best_params_,
            'results': gscv.cv_results_
            }
        end_time = time.time()
        execution_time = end_time - start_time
        print(f'O modelo {model} passou pela busca em grade em {int(execution_time//60)} minutos e {execution_time%60:.1f} segundos.\n')

    else:
        print(f'O modelo {model} está desativado.\n')

Iniciar lightgbm.


c:\PROJETOS NO GIT\Tsu\ethereum_laundering\.venv\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning:

invalid value encountered in cast



O modelo lightgbm passou pela busca em grade em 3 minutos e 25.4 segundos.

Iniciar random_forest.
O modelo random_forest passou pela busca em grade em 1 minutos e 30.1 segundos.

Iniciar decision_tree.
O modelo decision_tree passou pela busca em grade em 0 minutos e 1.1 segundos.

Iniciar knn.
O modelo knn passou pela busca em grade em 0 minutos e 3.4 segundos.

Iniciar catboost.
O modelo catboost passou pela busca em grade em 17 minutos e 3.1 segundos.

Iniciar xgboost.
[0]	validation_0-logloss:0.46012
[1]	validation_0-logloss:0.40913
[2]	validation_0-logloss:0.36921
[3]	validation_0-logloss:0.33568
[4]	validation_0-logloss:0.30687
[5]	validation_0-logloss:0.28247
[6]	validation_0-logloss:0.26105
[7]	validation_0-logloss:0.24256
[8]	validation_0-logloss:0.22665


c:\PROJETOS NO GIT\Tsu\ethereum_laundering\.venv\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning:

invalid value encountered in cast



[9]	validation_0-logloss:0.21250
[10]	validation_0-logloss:0.20021
[11]	validation_0-logloss:0.18884
[12]	validation_0-logloss:0.17751
[13]	validation_0-logloss:0.16821
[14]	validation_0-logloss:0.16001
[15]	validation_0-logloss:0.15233
[16]	validation_0-logloss:0.14540
[17]	validation_0-logloss:0.13929
[18]	validation_0-logloss:0.13264
[19]	validation_0-logloss:0.12699
[20]	validation_0-logloss:0.12202
[21]	validation_0-logloss:0.11762
[22]	validation_0-logloss:0.11338
[23]	validation_0-logloss:0.10966
[24]	validation_0-logloss:0.10596
[25]	validation_0-logloss:0.10276
[26]	validation_0-logloss:0.09962
[27]	validation_0-logloss:0.09690
[28]	validation_0-logloss:0.09313
[29]	validation_0-logloss:0.08992
[30]	validation_0-logloss:0.08717
[31]	validation_0-logloss:0.08529
[32]	validation_0-logloss:0.08273
[33]	validation_0-logloss:0.08071
[34]	validation_0-logloss:0.07900
[35]	validation_0-logloss:0.07747
[36]	validation_0-logloss:0.07586
[37]	validation_0-logloss:0.07450
[38]	validation

In [ ]:
for model, items in dct_final_hyperparams.items():
    print(f"{model}: {items['best_gscv']}\n")

lightgbm: {'learning_rate': 0.10000000000000002, 'max_depth': 21, 'n_estimators': 250, 'num_leaves': 18}

random_forest: {'max_depth': 26, 'max_leaf_nodes': 69, 'n_estimators': 300}

decision_tree: {'max_depth': 8, 'max_leaf_nodes': 29}

knn: {'n_neighbors': 31}

catboost: {'depth': 7, 'iterations': 250, 'l2_leaf_reg': 3.0, 'learning_rate': 0.09500000000000001}

xgboost: {'learning_rate': 0.09500000000000001, 'max_depth': 5, 'n_estimators': 300, 'reg_lambda': 4.5}



In [ ]:
df_search_results = pd.DataFrame()

for model in dct_final_hyperparams.keys():
    print(model)
    df_random_search_results = pd.DataFrame.from_dict(dct_final_hyperparams[model]['results'], orient='columns')

    df_random_search_results.insert(0, 'model', model)

    df_search_results = pd.concat([df_search_results, df_random_search_results], axis=0, ignore_index=True)

df_search_results.sort_values(by=['model', 'rank_test_roc_auc'], inplace=True, ascending=True)
display(df_search_results.loc[df_search_results['rank_test_roc_auc'] <= 3])


if GENERATE_REPORTS:
    df_search_results.to_excel(REPORTS_FOLDER + f'artigo_grid_search-{scoring_metric}.xlsx')
else:
    print(f'GENERATE_REPORTS: {GENERATE_REPORTS}')

lightgbm
random_forest
decision_tree
knn
catboost
xgboost


,model,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_num_leaves,params,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_train_roc_auc,split1_train_roc_auc,split2_train_roc_auc,mean_train_roc_auc,std_train_roc_auc,split0_test_f1,split1_test_f1,split2_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_train_f1,split1_train_f1,split2_train_f1,mean_train_f1,std_train_f1,split0_test_recall,split1_test_recall,split2_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_train_recall,split1_train_recall,split2_train_recall,mean_train_recall,std_train_recall,param_max_leaf_nodes,param_n_neighbors,param_depth,param_iterations,param_l2_leaf_reg,param_reg_lambda
349,catboost,10.26053,0.09321,0.06608,0.00725,0.09500,NaN,NaN,NaN,"{'depth': 7, 'iterations': 250, 'l2_leaf_reg': 3.0, 'learning_rate': 0.09500000000000001}",0.99824,0.99881,0.99779,0.99828,0.00042,1,1.00000,1.00000,1.00000,1.00000,0.00000,0.96529,0.97862,0.96473,0.96954,0.00642,12,1.00000,1.00000,1.00000,1.00000,0.00000,0.94652,0.96434,0.95300,0.95462,0.00737,38,1.00000,1.00000,1.00000,1.00000,0.00000,NaN,NaN,7.00000,250.00000,3.00000,NaN
449,catboost,34.99455,1.18058,0.13642,0.02334,0.09500,NaN,NaN,NaN,"{'depth': 9, 'iterations': 250, 'l2_leaf_reg': 3.5, 'learning_rate': 0.09500000000000001}",0.99849,0.99872,0.99758,0.99826,0.00049,2,1.00000,1.00000,1.00000,1.00000,0.00000,0.96603,0.98030,0.96314,0.96982,0.00750,6,1.00000,1.00000,1.00000,1.00000,0.00000,0.94489,0.96759,0.95300,0.95516,0.00939,22,1.00000,1.00000,1.00000,1.00000,0.00000,NaN,NaN,9.00000,250.00000,3.50000,NaN
433,catboost,29.29753,0.61419,0.14365,0.02653,0.09500,NaN,NaN,NaN,"{'depth': 9, 'iterations': 200, 'l2_leaf_reg': 3.5, 'learning_rate': 0.09500000000000001}",0.99845,0.99879,0.99752,0.99826,0.00054,3,1.00000,1.00000,1.00000,1.00000,0.00000,0.96694,0.97865,0.96230,0.96930,0.00688,20,1.00000,1.00000,1.00000,1.00000,0.00000,0.94814,0.96596,0.95138,0.95516,0.00775,22,1.00000,1.00000,1.00000,1.00000,0.00000,NaN,NaN,9.00000,200.00000,3.50000,NaN
251,decision_tree,0.09077,0.01370,0.01546,0.00156,NaN,8.00000,NaN,NaN,"{'max_depth': 8, 'max_leaf_nodes': 29}",0.98104,0.98254,0.98417,0.98258,0.00128,1,0.99079,0.99000,0.99031,0.99037,0.00033,0.91653,0.92129,0.90998,0.91594,0.00464,5,0.94814,0.94629,0.95659,0.95034,0.00449,0.92545,0.90113,0.90113,0.90924,0.01146,1,0.94814,0.93517,0.94652,0.94327,0.00577,29.00000,NaN,NaN,NaN,NaN,NaN
250,decision_tree,0.07801,0.00733,0.01388,0.00023,NaN,8.00000,NaN,NaN,"{'max_depth': 8, 'max_leaf_nodes': 28}",0.98099,0.98213,0.98429,0.98247,0.00137,2,0.99073,0.98959,0.99025,0.99019,0.00047,0.91566,0.91833,0.90805,0.91401,0.00436,6,0.94643,0.94006,0.95531,0.94727,0.00625,0.92382,0.89303,0.89627,0.90438,0.01382,2,0.94489,0.92139,0.94408,0.93679,0.01089,28.00000,NaN,NaN,NaN,NaN,NaN
248,decision_tree,0.10486,0.01369,0.01809,0.00541,NaN,8.00000,NaN,NaN,"{'max_depth': 8, 'max_leaf_nodes': 26}",0.98091,0.98182,0.98347,0.98207,0.00106,3,0.98966,0.98675,0.98975,0.98872,0.00139,0.91512,0.91937,0.90507,0.91319,0.00599,7,0.94415,0.93899,0.95258,0.94524,0.00560,0.91734,0.89627,0.89627,0.90330,0.00993,4,0.93841,0.92301,0.94408,0.93517,0.00890,26.00000,NaN,NaN,NaN,NaN,NaN
256,knn,0.00853,0.00071,0.89683,0.06755,NaN,NaN,NaN,NaN,{'n_neighbors': 31},0.92352,0.93742,0.92028,0.92707,0.00743,1,0.94150,0.93707,0.94404,0.94087,0.00288,0.70862,0.73466,0.71416,0.71915,0.01120,2,0.74045,0.72244,0.73825,0.73371,0.00802,0.66613,0.68882,0.66613,0.67369,0.01070,1,0.69125,0.67180,0.69368,0.68558,0.00979,NaN,31.00000,NaN,NaN,NaN,NaN
257,knn,0.00752,0.00041,0.93637,0.05960,NaN,NaN,NaN,NaN,{'n_neighbors': 32},0.92348,0.93709,0.92046,0.92701,0.00723,2,0.94121,0.93645,0.94372,0.94046,0.00301,0.71018,0.73016,0.70888,0.71641,0.00974,4,0.73707,0.71686,0.73109,0.72834,0.00848,0.66126,0.67099,0.65316,0.66180,0.00729,4,0.68152,0.65964,0.67747,0.67288,0.00950,NaN,32.00000,NaN,N